In [1]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
import re
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

kkma = Kkma()

/Users/heeseok/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


얘는 date로 최근거만 불러와서 data frame으로

In [2]:
import pandas as pd
data = pd.read_csv('total_data.csv')
len(data)

114880

In [5]:
data_ = data[[type(_)==str for _ in data.controlled_collection]]

In [3]:
termLex = pd.read_csv('dictionary.csv')

Dictionary = defaultdict(lambda:-1)
for idx,pos in enumerate(termLex.dictionary):
    Dictionary[idx]=pos
    
V = termLex.vocabulary.to_list()

In [11]:
from sklearn.cluster import KMeans

In [181]:
data2019 = data_[[_.startswith('2019-') for _ in data_.date]]
tfidf = TfidfVectorizer(vocabulary=V,sublinear_tf=True)
tf_idf_matrix = tfidf.fit_transform(data2019.controlled_collection)

In [228]:
data201904 = data2019[[_.startswith('2019-04') for _ in data2019.date]]
tfidf = TfidfVectorizer(vocabulary=V,sublinear_tf=True)
tf_idf_matrix = tfidf.fit_transform(data201904.controlled_collection)

In [289]:
kmeans = KMeans(n_clusters = 10).fit(tf_idf_matrix)

In [317]:
clusters=defaultdict(list)
p = re.compile(r'민원|관련|경우|해당|사항|내용|문의|불편|진행|피해|요청|확인|이용|설치|개최|주민|처리|나라|생각|여부|연락|주신|구정|구청|시간')
    

for i in range(len(data201904)):
    clusters[kmeans.labels_[i]].append(''.join(p.sub('',data201904.iloc[i].controlled_collection)))

temp = [(_[0],len(_[1])) for _ in clusters.items()]

for i in sorted(temp,key=lambda x:x[1], reverse=True):
    fuck = ' '.join(clusters[i[0]])
    wordcloud = WordCloud(font_path='/Library/Fonts/AppleGothic.ttf',background_color='white').generate(fuck)
    print(' '.join([_[0] for _ in [(_[0],_[1]) for _ in wordcloud.words_.items()][5::-1]]))

사업 공무원 서울시 직원 건물 공사
과태료 부과 주차 단속 도로 불법 주차 차량 단속
도로 마을 버스 차량 사업 주차장 주차
부탁 운영 공사 센터 도서관 도로
조치 작업 발생 현장 소음 공사
부탁 주시 안전 지역 의견 공원
사진 조치 의견 현수막 관심 신고
사업 시행 조합 임원 고덕 단지 주택 건축 정비 사업비 총회
쓰레기 무단 청소 배출 무단 투기 수거 쓰레기
금연 흡연자 간접 흡연 흡연 단속 금연 구역


In [ ]:
#LDA  alpah 1이상  베타는 0.1미만